In [5]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import optuna
import os
import json 


conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
    SELECT DISTINCT
        f.match_date,
        f.team,
        f.is_home,
        f.gw,
        f.[is_promoted?],
        f.[is_early_season?],
        f.season,
        f.division,
        team_elo.elo as team_elo,
        f.team_rolling_summary_goals,
        f.team_rolling_summary_pens_made,
        f.team_rolling_summary_shots,
        f.team_rolling_summary_shots_on_target,
        f.team_rolling_summary_touches,
        f.team_rolling_summary_xg,
        f.team_rolling_summary_npxg,
        f.team_rolling_possession_touches_att_pen_area,
        f.team_rolling_keeper_psxg,
        f.team_rolling_conceded_summary_goals,
        f.team_rolling_conceded_summary_pens_made,
        f.team_rolling_conceded_summary_shots,
        f.team_rolling_conceded_summary_shots_on_target,
        f.team_rolling_conceded_summary_touches,
        f.team_rolling_conceded_summary_xg,
        f.team_rolling_conceded_summary_npxg,
        f.team_rolling_conceded_possession_touches_att_pen_area,
        f.team_rolling_conceded_keeper_psxg,
        f.opp_team,
        opp_elo.elo as opp_team_elo,
        f.opp_team_rolling_summary_goals,
        f.opp_team_rolling_summary_pens_made,
        f.opp_team_rolling_summary_shots,
        f.opp_team_rolling_summary_shots_on_target,
        f.opp_team_rolling_summary_touches,
        f.opp_team_rolling_summary_xg,
        f.opp_team_rolling_summary_npxg,
        f.opp_team_rolling_possession_touches_att_pen_area,
        f.opp_team_rolling_keeper_psxg,
        f.opp_team_rolling_conceded_summary_goals,
        f.opp_team_rolling_conceded_summary_pens_made,
        f.opp_team_rolling_conceded_summary_shots,
        f.opp_team_rolling_conceded_summary_shots_on_target,
        f.opp_team_rolling_conceded_summary_touches,
        f.opp_team_rolling_conceded_summary_xg,
        f.opp_team_rolling_conceded_summary_npxg,
        f.opp_team_rolling_conceded_possession_touches_att_pen_area,
        f.opp_team_rolling_conceded_keeper_psxg,
        ms.summary_goals as goals
    FROM 
        team_all_features_90_00200 f
    JOIN
        fbref_match_all_columns ms
            ON ms.match_url = f.match_url
            AND ms.team = f.team
    JOIN
        clubelo_features team_elo
            ON team_elo.fbref_team = f.team
                AND DATE(f.match_date) BETWEEN team_elo.start_date AND team_elo.end_date
    JOIN
        clubelo_features opp_elo
            ON opp_elo.fbref_team = f.opp_team
                AND DATE(f.match_date) BETWEEN opp_elo.start_date AND opp_elo.end_date
    WHERE 
        f.team_rolling_summary_goals IS NOT NULL
        AND f.division = 'Premier League'
                       """, conn)

df

,match_date,team,is_home,gw,is_promoted?,is_early_season?,season,division,team_elo,team_rolling_summary_goals,...,opp_team_rolling_conceded_summary_goals,opp_team_rolling_conceded_summary_pens_made,opp_team_rolling_conceded_summary_shots,opp_team_rolling_conceded_summary_shots_on_target,opp_team_rolling_conceded_summary_touches,opp_team_rolling_conceded_summary_xg,opp_team_rolling_conceded_summary_npxg,opp_team_rolling_conceded_possession_touches_att_pen_area,opp_team_rolling_conceded_keeper_psxg,goals
0,2025-05-25 00:00:00,Tottenham,1,38,0,0,2024-2025,Premier League,1778.323242,0.811222,...,1.604323,0.189265,12.478522,4.128437,603.795044,1.482446,1.331035,26.528746,1.392191,1.0
1,2025-05-25 00:00:00,Brighton,0,38,0,0,2024-2025,Premier League,1809.790894,1.942928,...,2.199083,0.000000,15.406190,5.449610,593.018515,1.694755,1.694755,26.197995,2.080640,4.0
2,2025-05-25 00:00:00,Bournemouth,1,38,0,0,2024-2025,Premier League,1806.686401,1.221826,...,1.501753,0.165470,17.459047,4.194789,654.869872,1.860305,1.690205,28.673113,1.399942,2.0
3,2025-05-25 00:00:00,Leicester City,0,38,1,0,2024-2025,Premier League,1620.328247,1.052791,...,1.662430,0.099022,11.401362,4.082394,605.052197,1.304388,1.225170,23.430866,1.373387,0.0
4,2025-05-25 00:00:00,Newcastle Utd,1,38,0,0,2024-2025,Premier League,1878.094604,1.851561,...,0.968051,0.000000,11.809791,4.587958,650.468474,1.138399,1.138399,22.299200,1.289435,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5330,2020-09-12 00:00:00,Leeds United,0,1,1,1,2020-2021,Premier League,1624.624268,2.418047,...,1.436920,0.095730,7.266258,2.635221,496.180466,1.205448,1.128864,10.108849,1.160007,3.0
5331,2020-09-12 00:00:00,Crystal Palace,1,1,0,1,2020-2021,Premier League,1673.688843,0.570460,...,0.822600,0.000000,10.651285,3.048056,544.952012,1.216448,1.216448,20.837488,1.270361,1.0
5332,2020-09-12 00:00:00,Southampton,0,1,0,1,2020-2021,Premier League,1720.211182,1.923482,...,1.970332,0.000000,12.669481,4.397582,686.387950,1.478836,1.478836,20.791894,1.788755,0.0
5333,2020-09-12 00:00:00,Fulham,1,1,1,1,2020-2021,Premier League,1591.546265,1.364007,...,1.193816,0.199407,13.800604,4.952256,574.073749,1.695469,1.538354,24.615318,1.463729,0.0


In [6]:
model = xgb.Booster()
model.load_model('best_xgb_model_prem_simple.json')
print("Model loaded successfully!")

Model loaded successfully!


In [7]:
X = df.drop(columns=["team", "opp_team", "match_date", "goals"])
X = pd.get_dummies(X, columns=["season", "division"], drop_first=True)

X = X[(X["gw"] == 38) & X["season_2024-2025"] == True]

dmatrix = xgb.DMatrix(X)

predictions = model.predict(dmatrix)

X

,is_home,gw,is_promoted?,is_early_season?,team_elo,team_rolling_summary_goals,team_rolling_summary_pens_made,team_rolling_summary_shots,team_rolling_summary_shots_on_target,team_rolling_summary_touches,...,opp_team_rolling_conceded_summary_npxg,opp_team_rolling_conceded_possession_touches_att_pen_area,opp_team_rolling_conceded_keeper_psxg,season_2018-2019,season_2019-2020,season_2020-2021,season_2021-2022,season_2022-2023,season_2023-2024,season_2024-2025
0,1,38,0,0,1778.323242,0.811222,0.115187,9.406658,2.905519,561.733668,...,1.331035,26.528746,1.392191,False,False,False,False,False,False,True
1,0,38,0,0,1809.790894,1.942928,0.413991,13.373102,5.344124,611.614848,...,1.694755,26.197995,2.080640,False,False,False,False,False,False,True
2,1,38,0,0,1806.686401,1.221826,0.000000,12.260395,3.459288,577.876160,...,1.690205,28.673113,1.399942,False,False,False,False,False,False,True
3,0,38,1,0,1620.328247,1.052791,0.000000,8.921843,2.994827,564.093939,...,1.225170,23.430866,1.373387,False,False,False,False,False,False,True
4,1,38,0,0,1878.094604,1.851561,0.186736,13.642049,4.539317,584.807938,...,1.138399,22.299200,1.289435,False,False,False,False,False,False,True
5,0,38,0,0,1779.701660,1.316445,0.072671,9.607141,3.648463,540.980054,...,0.904099,23.016945,1.087742,False,False,False,False,False,False,True
6,1,38,0,0,1786.741577,1.467455,0.000000,13.951470,4.743678,604.152584,...,0.844689,15.539044,0.678824,False,False,False,False,False,False,True
7,0,38,0,0,1945.100342,1.746199,0.047870,14.814098,4.701548,764.003099,...,1.311028,23.687600,1.567378,False,False,False,False,False,False,True
8,1,38,0,0,1807.889526,1.317446,0.000000,11.261761,4.096023,517.522092,...,0.700628,18.124731,0.689590,False,False,False,False,False,False,True
9,0,38,0,0,1885.077271,1.416079,0.143912,15.270735,5.445411,619.968575,...,1.421867,28.425949,1.298327,False,False,False,False,False,False,True


In [9]:
import pandas as pd
import numpy as np
import xgboost as xgb
from itertools import combinations

def create_power_rankings(df, model, gw=38, season="2024-2025"):
    """
    Create power rankings by simulating all teams playing each other home and away
    """
    
    print(f"Creating power rankings for GW {gw}, Season {season}")
    
    # Step 1: Filter to only teams that actually exist in the target season and gameweek
    season_data = df[(df['season'] == season) & (df['gw'] == gw)].copy()
    unique_teams = season_data['team'].unique()
    print(f"Found {len(unique_teams)} teams in {season}: {list(unique_teams)}")
    
    # Step 2: Create all possible matchups (home and away)
    matchups = []
    teams_list = list(unique_teams)
    
    for home_team in teams_list:
        for away_team in teams_list:
            if home_team != away_team:  # Don't play against themselves
                matchups.append({
                    'home_team': home_team,
                    'away_team': away_team
                })
    
    print(f"Created {len(matchups)} total matchups")
    
    # Step 3: Get expected season columns from model
    expected_features = model.feature_names
    expected_season_cols = [col for col in expected_features if col.startswith('season_')]
    print(f"Expected season columns: {expected_season_cols}")
    
    # Step 4: Create predictions for each matchup
    all_results = []
    
    for matchup in matchups:
        home_team = matchup['home_team']
        away_team = matchup['away_team']
        
        # Get home team data (they are "team") - using season_data to ensure correct season
        home_data = season_data[season_data['team'] == home_team].copy()
        
        if len(home_data) == 0:
            print(f"Warning: No data for {home_team} at GW {gw} in {season}")
            continue
            
        # Take first row and set opponent to away team
        match_data = home_data.iloc[0:1].copy()
        match_data['opp_team'] = away_team
        match_data['is_home'] = 1  # Home team
        
        # Get away team's stats to use as opponent features
        away_data = season_data[season_data['team'] == away_team].copy()
        
        if len(away_data) == 0:
            print(f"Warning: No data for {away_team} at GW {gw} in {season}")
            continue
            
        away_stats = away_data.iloc[0]
        
        # Replace opp_team features with away team's actual features
        for col in match_data.columns:
            if col.startswith('opp_team_'):
                corresponding_col = col.replace('opp_team_', 'team_')
                if corresponding_col in away_stats.index:
                    match_data[col] = away_stats[corresponding_col]
        
        # Preprocessing - drop non-feature columns
        X_match = match_data.drop(columns=["team", "opp_team", "match_date"])
        
        # Create dummy variables for season
        X_match = pd.get_dummies(X_match, columns=["season"], drop_first=True)
        
        # Ensure all expected season columns are present
        for season_col in expected_season_cols:
            if season_col not in X_match.columns:
                X_match[season_col] = 0
        
        # Set the correct season to 1
        target_season_col = f"season_{season}"
        if target_season_col in X_match.columns:
            X_match[target_season_col] = 1
        
        # Filter by gameweek
        X_match = X_match[X_match["gw"] == gw]
        
        # Ensure columns are in the same order as expected by model
        X_match = X_match.reindex(columns=expected_features, fill_value=0)
        
        # Predict home goals
        dmatrix = xgb.DMatrix(X_match)
        home_goals = model.predict(dmatrix)[0]
        
        # Now predict away goals (swap teams)
        away_match_data = away_data.iloc[0:1].copy()
        away_match_data['opp_team'] = home_team
        away_match_data['is_home'] = 1  # Away team becomes "home" for their prediction
        
        # Replace opp_team features with home team's stats
        home_stats = home_data.iloc[0]
        for col in away_match_data.columns:
            if col.startswith('opp_team_'):
                corresponding_col = col.replace('opp_team_', 'team_')
                if corresponding_col in home_stats.index:
                    away_match_data[col] = home_stats[corresponding_col]
        
        # Same preprocessing for away prediction
        X_away_match = away_match_data.drop(columns=["team", "opp_team", "match_date"])
        X_away_match = pd.get_dummies(X_away_match, columns=["season"], drop_first=True)
        
        # Ensure all expected season columns are present
        for season_col in expected_season_cols:
            if season_col not in X_away_match.columns:
                X_away_match[season_col] = 0
        
        # Set the correct season to 1
        if target_season_col in X_away_match.columns:
            X_away_match[target_season_col] = 1
            
        # Filter by gameweek
        X_away_match = X_away_match[X_away_match["gw"] == gw]
        
        # Ensure columns are in the same order as expected by model
        X_away_match = X_away_match.reindex(columns=expected_features, fill_value=0)
        
        dmatrix_away = xgb.DMatrix(X_away_match)
        away_goals = model.predict(dmatrix_away)[0]
        
        all_results.append({
            'home_team': home_team,
            'away_team': away_team,
            'home_goals': home_goals,
            'away_goals': away_goals,
            'goal_difference': home_goals - away_goals
        })
    
    complete_df = pd.DataFrame(all_results)
    
    # Step 5: Calculate power rankings with per-90 stats
    team_stats = {}
    
    for team in teams_list:
        # Home matches
        home_matches = complete_df[complete_df['home_team'] == team]
        # Away matches  
        away_matches = complete_df[complete_df['away_team'] == team]
        
        # Calculate total goals
        total_goals_for = (home_matches['home_goals'].sum() + 
                          away_matches['away_goals'].sum())
        total_goals_against = (home_matches['away_goals'].sum() + 
                             away_matches['home_goals'].sum())
        
        total_matches = len(home_matches) + len(away_matches)
        
        # Calculate per-90 stats (assuming each match is 90 minutes)
        goals_for_per_90 = total_goals_for / total_matches if total_matches > 0 else 0
        goals_against_per_90 = total_goals_against / total_matches if total_matches > 0 else 0
        goal_difference_per_90 = goals_for_per_90 - goals_against_per_90
        
        team_stats[team] = {
            'team': team,
            'matches_played': total_matches,
            'goals_for_per_90': round(goals_for_per_90, 3),
            'goals_against_per_90': round(goals_against_per_90, 3),
            'goal_difference_per_90': round(goal_difference_per_90, 3)
        }
    
    # Create final rankings DataFrame
    rankings_df = pd.DataFrame(list(team_stats.values()))
    rankings_df = rankings_df.sort_values(['goal_difference_per_90', 'goals_for_per_90'], 
                                        ascending=[False, False])
    rankings_df['rank'] = range(1, len(rankings_df) + 1)
    
    # Reorder columns
    rankings_df = rankings_df[['rank', 'team', 'matches_played', 'goals_for_per_90', 
                             'goals_against_per_90', 'goal_difference_per_90']]
    
    return rankings_df, complete_df

# Usage
print("Generating power rankings...")
power_rankings, all_results = create_power_rankings(df, model, gw=38, season="2024-2025")

print("\n" + "="*80)
print("POWER RANKINGS")
print("="*80)
print(power_rankings.to_string(index=False))

print(f"\nTotal simulated matches: {len(all_results)}")
print(f"Teams analyzed: {len(power_rankings)}")

# Show some example matchup predictions
print("\n" + "="*50)
print("SAMPLE PREDICTED RESULTS")
print("="*50)
sample_results = all_results
for _, match in sample_results.iterrows():
    print(f"{match['home_team']} {match['home_goals']:.1f} - {match['away_goals']:.1f} {match['away_team']}")

Generating power rankings...
Creating power rankings for GW 38, Season 2024-2025
Found 20 teams in 2024-2025: ['Tottenham', 'Brighton', 'Bournemouth', 'Leicester City', 'Newcastle Utd', 'Everton', 'Fulham', 'Manchester City', "Nott'ham Forest", 'Chelsea', 'Liverpool', 'Crystal Palace', 'Southampton', 'Arsenal', 'Wolves', 'Brentford', 'Ipswich Town', 'West Ham', 'Manchester Utd', 'Aston Villa']
Created 380 total matchups
Expected season columns: ['season_2018-2019', 'season_2019-2020', 'season_2020-2021', 'season_2021-2022', 'season_2022-2023', 'season_2023-2024', 'season_2024-2025']

POWER RANKINGS
 rank            team  matches_played  goals_for_per_90  goals_against_per_90  goal_difference_per_90
    1 Manchester City              38             2.331                 1.062                   1.270
    2         Chelsea              38             1.930                 1.173                   0.757
    3         Arsenal              38             2.058                 1.444           